<a href="https://cocl.us/Data_Science_with_Scalla_top"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/SC0103EN/adds/Data_Science_with_Scalla_notebook_top.png" width = 750, align = "center"></a>
 <br/>
<a><img src="https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width="200" align="center"></a>"

# 3.4.1 Decision Trees

After completing this lesson you should be able to:

* Understand the pipelines API for decision trees
* Describe Pipeline's input and output columns 
* Perform classification and regression with decision trees 
* Understand and use decision trees parameters 

## Decision Trees 

A decision tree is an effective non-parametric modeling method for different types of data. It can handle both continuous and categorical data. It can also help us identify markers that split our categories. 

* Popular method for classification and regression
* Easy to interpret
* Handle categorical features
* Extend to multiclass classification
* Do NOT require feature scaling
* They capture non-linearities and feature interactions

## MLlib's implementation 

MLlib's implementation of Decision Trees supports both binary and multi-class classification as well as regression. It also supports both continuous and categorical features. In MLlib the data is partitioned by rows allowing distributed training of Decision Trees. 

* Supports binary and multiclass classification
* Supports regression
* Supports continuous and categorical features 
* Partitions data by rows, allowing distributed training
* Used by the Pipelines API for Decision Trees

This implementation is then used by the Pipelines API for Decision Trees which I'll introduce in this lesson. 

## Spark.ML API for Decision Trees

* More functionalities in the original MLlib API 
* Separation of Decision Trees for classification and regression
* Use of DF metadata to distinguish between continuous and categorical features
* For classification trees
	* class conditional probabilities, that is, predicted probabilities of each class, made available
	* estimates of feature importance


The Spark.ML API for decision trees contains more functionalities in the original MLlib API.

There is a separation of decision trees the classification and regression.

We use DataFrames meta data to distinguish between continuous and categorical features 

## Inputs and Outputs 

Now let's take a look at the inputs taken and the output produced by decision trees in the Pipelines API. As inputs, there are the label and feature columns. If its corresponding parameter names are omitted the API will look for its defaults, label and features in the import DataFrame. As outputs there are the prediction, rawPrediction and probability columns, where the last two only apply for classification trees. 

If its corresponding parameter names are omitted the API will append these columns with its default names to the resulting DataFrame.

Notice that if I don't want to give a column to be appended I just have to set its name as an empty string using the appropriate parameter name. 

## Loading Data

In [1]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()
import spark.implicits._
import org.apache.spark.sql.functions._

Intitializing Scala interpreter ...

Spark Web UI available at http://host.docker.internal:4040
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1670053347795)
SparkSession available as 'spark'


22/12/03 15:42:38 WARN ProcfsMetricsGetter: Exception when trying to compute pagesize, as a result reporting of ProcessTree metrics is stopped


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@27587d66
import spark.implicits._
import org.apache.spark.sql.functions._


In [2]:
import org.apache.spark.ml.classification.DecisionTreeClassifier
import org.apache.spark.ml.classification.DecisionTreeClassificationModel

import org.apache.spark.ml.classification.DecisionTreeClassifier
import org.apache.spark.ml.classification.DecisionTreeClassificationModel


In [ ]:
import sys.process._
"wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/SC0105EN/data/sample_libsvm_data.txt  -P /resources/data/"!

In [5]:
val dtC = new DecisionTreeClassifier().setLabelCol("indexedLabel").setFeaturesCol("indexedFeatures")

import org.apache.spark.ml.Pipeline
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.ml.feature.{StringIndexer, IndexToString, VectorIndexer}
import org.apache.spark.mllib.util.MLUtils.{
  convertVectorColumnsFromML => fromML,
  convertVectorColumnsToML => toML
}

val data = toML(MLUtils.loadLibSVMFile(sc, "/resources/data/sample_libsvm_data.txt").toDF)
data.show()

java.lang.IllegalArgumentException:  Invalid start or len parameter

This is a sample data set that comes with Spark. If you didn't get Spark from sources, then you probably don't have it. From a Spark notebook you can get it from the shell command `wget`; remember this will not work on Microsoft Windows boxes. Then we can load the data into a `DataFrame` using the `loadLibSVMFile()` function and then calling `toDF` on it. 

## Creating the Tree Model

In [ ]:
val labelIndexer = new StringIndexer().setInputCol("label").setOutputCol("indexedLabel").fit(data)
val labelConverter = new IndexToString().setInputCol("prediction").setOutputCol("predictedLabel").setLabels(labelIndexer.labels)
val featureIndexer = new VectorIndexer().setInputCol("features").setOutputCol("indexedFeatures").setMaxCategories(4).fit(data)
val pipelineClass = new Pipeline().setStages(Array(labelIndexer, featureIndexer, dtC, labelConverter))
val Array(trainingData, testData) = data.randomSplit(Array(0.7, 0.3))

Next we set up our input and output columns and then fit the data for the `labelIndexer`. From that, we get the `labelConverter` from it. Then we set up a `featureIndexer` from `VectorIndexer`, which will perform a fit call on with our input data. Next we set up the pipeline of the data with the stages are derived from those values we just created and perform a random split to get our `trainingData` and `testData` values. 

## DecisionTreeClassifier I

In [ ]:
val modelClassifier = pipelineClass.fit(trainingData)
val treeModel = modelClassifier.stages(2).asInstanceOf[DecisionTreeClassificationModel]
println("Learned classification tree model:\n" + treeModel.toDebugString)

To get our `DecisionTreeClassifier` we create a `ModelClassifier` by fitting our pipeline from the `trainingData` values and then we create the `treeModels` by casting the data to an instance of a Decision Tree Classification model and then print out the data.

## DecisionTreeClassifier II

In [ ]:
// Getting results from the model
val predictionsClass = modelClassifier.transform(testData)
predictionsClass.show()

You may notice that the `Indexedlabel` is presented as 1, while label suggests 0, and vice versa. 

Please note the by  passing the model by the String Indexer and label converter, Spark has signed its own alias to the labels provided, hence the prediction is also represented in Spark's own alias. The predicted label, however, returns a string value that represents the original names of the labels in the dataset.

## DecisionTreeRegressor I

In [ ]:
import org.apache.spark.ml.regression.DecisionTreeRegressor
import org.apache.spark.ml.regression.DecisionTreeRegressionModel

val dtR = new DecisionTreeRegressor().setLabelCol("label").setFeaturesCol("indexedFeatures")

val pipelineReg = new Pipeline().setStages(Array(featureIndexer, dtR))

The same goes for the decision tree `regressor`. In the case of regression, remember that my Pipeline had only two stages: the features indexer and the decision tree `regressor`. 

## DecisionTreeRegressor II

In [ ]:
val modelRegressor = pipelineReg.fit(trainingData)

val treeModel = modelRegressor.stages(1).asInstanceOf[DecisionTreeRegressionModel]

println("Learned regression tree model:\n" + treeModel.toDebugString)

Similarly to a `DecisionTreeClassifier` we get a `DecisionTreeRegressor` by creating a `modelRegressor` value from fitting our pipeline from the `trainingData` values. And, like before, we create the `treeModels` by casting the data to an instance of a Decision Tree Regression Model, and print out the data. 

## DecisionTreeRegressor III

In [ ]:
// Getting results from the model
val predictionsReg = modelRegressor.transform(testData)
predictionsReg.show()

Getting results from the model we have a printout of the data. 

## Problem Specification Parameters

* Describe the problem and the dataset
* Should be specified
* Do not require tuning
* Parameters:
	* `numClasses`: number of classes (classification)
	* `categoricalFeaturesInfo`: specifies which features are categorical and how many categorical values each of those features can take
		* optional: if not specified, alorithm may still get reasonable results
		* BUT performance should be better if categorical features are designated
		* map from feature indices to number of categories
		* features not in the map are treated as continuous

Now let's take a look into all different parameters of Decision Trees. 

The parameters can be classified into three categories: problem specification, stopping criteria and tunable parameters. 

I'll start with the problem specification parameters. As their name suggests they describe the problem and the dataset. They should be properly specified and do not require tuning. This category of parameters include: 

`numClasses`, the number of classes in a classification problem. 

`categoricalFeaturesInfo` is an optional parameter that specifies which features are categorical and how many categorical values each of those features can take.  Although it is an optional parameter, if specified properly it can lead to better performance. 

## Stopping Criteria

* Determine when the tree stops building
* May lead to overfitting
* Need to be validate on held-out test data

The parameters in the stopping criteria category determine when the tree stops building. The specification of these parameters need to be validated on held-out test data, as they may lead to overfitting. 

## Stopping Criteria, Parameters 

* `maxDepth`: maximum depth of a tree
	* if it increases (deeper trees):
		* more expressive, potentially higher accuracy
		* more costly to train
		* more likely to overfit
* `minInstancesPerNode`: each child must receive at least this number of instances for a node to be split further
	* commonly used in Random Forests as its trees are deeper and may overfit
* `minInfoGain`: the split must improve this much, in terms of information gain, for a node to be split further
	* The information gain is the difference between the parent node impurity and the weighted sum of the two child node impurities
	* Node impurity is a measure of the homogeniety of the labels at the node

This category of parameters include: `maxDepth`, the maximum depth of the tree. If this parameter is increased, that is, if it is set to build deeper trees, the resulting model may be more expressive and more accurate, but will also be more costly to train and more likely to overfit.

`minInstancesPerNode`, the minimum number of instances the children node must receive for a node to be split further. It is commonly used in Random Forests; as trees go deeper and may overfit, resulting leaf nodes with only one instance each.  

`minInfoGain`, the minimum information gain obtained with a split for the node to be split further. The information gain is the difference between the parent node impurity and the weighted sum the two child nodes' impurities. The impurity of a node is a measure of the homogeneity of the labels at that particular node.

## Tunable Parameters I

* `maxBins`: number of bins used when discretizing continuous features
	* must be at least the maximum number of categories for any categorical feature
	* if it increases: 
		* allows the consideration of more split candidates and fine-grained split decisions
		* increases computation and communication

Finally, the tunable parameters. As their name suggests these are parameters for fine-tuning a model and they also need to be validated on held-out test data as they, too, may lead to overfitting. 

This final category of parameters include: maxBins; the number of bins used when discretizing continuous features, being equal or greater than the maximum number of categories for any categorical feature.

## Tunable Parameters II

* `maxMemoryInMB`: amount of memory to be used for collecting sufficient statistics
	* default = 256 MB, works in most scenarios
	* if it increases: 
		* can lead to faster training by allowing fewer passes over the data
		* there may be decreasing returns since amount of communication on each interaction also increases

`maxMemoryInMB`: the amount of memory to be used for collecting sufficient statistics. Its default stands at 256 megabytes which will work in most scenarios. If increased, this parameter can lead to faster training, since it allows fewer passes over the data. On the other hand, if increased too much, it may have decreasing returns as the amount of communication on each interaction also increases. 

## Tunable Parameters III

* `subsamplingRate`: fraction of the training data used for learning the decision tree
	* more relevant for training ensemblers of trees (see next Lesson)
* `impurity`: impurity measure used to choose between candidate splits
	* classification: Gini Impurity and Entropy
	* regression: Variance

`subSamplingRate`, the fraction of the training data used for learning the decision trade. This parameter is more relevant for training ensembles of trees like Random Forests.

`impurity`, the impurity measure used to choose between candidate splits. For classification problems, Gini Impurity and Entropy measures are supported. For regression problems the only supported measure is the Variance.

## Lesson Summary

Having completed this lesson, we should now be able to:

* Understand the Pipelines API for Decision Trees 
* Describe Pipeline's default Input and Output columns
* Perform classification and regression with Decision Trees
* Understand and use Decision Tree's parameters

### About the Authors

[Petro Verkhogliad](https://www.linkedin.com/in/vpetro) is Consulting Manager at Lightbend. He holds a Masters degree in Computer Science with specialization in Intelligent Systems. He is passionate about functional programming and applications of AI.